In [1]:
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from langchain import LLMChain, PromptTemplate
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import uuid
import uvicorn
import nest_asyncio

# Apply nest_asyncio for running in Jupyter
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Initialize Qdrant clientF
client = QdrantClient("localhost", port=6333)

#sentence-transformers/multi-qa-mpnet-base-dot-v1
#sentence-transformers/all-mpnet-base-v2
MODEL = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Load data and create collection
df = pd.read_csv('output_embeddings.csv')
text_column_name = 'text'

# Recreate collection in Qdrant
client.recreate_collection(
    collection_name="similar_text",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

# Insert data into Qdrant
for index, row in df.iterrows():
    text = row[text_column_name]
    text_embeddings = MODEL.encode(text).tolist()
    id = str(uuid.uuid4())
    payload = {"text": text, "text_embeddings": text_embeddings}
    client.upsert(
        collection_name="similar_text",
        wait=True,
        points=[PointStruct(id=id, vector=text_embeddings, payload=payload)]
    )

# Define the input schema for similarity search
class Query(BaseModel):
    question: str

@app.post("/search")
def search_similar_text(query: Query):
    question_embeddings = MODEL.encode(query.question).tolist()
    search_result = client.search(
        collection_name="similar_text",
        query_vector=question_embeddings,
        limit=3
    )
    results = [
        {"text": result.payload["text"], "similarity_score": result.score}
        for result in search_result
    ]
    return results

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

C:\Users\thana\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:     Started server process [18272]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57373 - "POST /search HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18272]


KeyboardInterrupt: 

In [6]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

import uvicorn

app = FastAPI()

# Initialize SentenceTransformer for embeddings
MODEL = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Initialize Qdrant client
client = QdrantClient(url="http://localhost:6333")

# Initialize GPT-Neo model
model_name = "EleutherAI/gpt-neo-2.7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

# Text generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.3,
    repetition_penalty=1.5
)

# Wrap pipeline with HuggingFacePipeline for LangChain
huggingface_llm = HuggingFacePipeline(pipeline=generator)

# Define LangChain prompt
prompt = PromptTemplate(
    template=(
        "Based on the following context, answer the question concisely without repeating the context: "
        "Context: {context} "
        "Question: {question} "
        "Answer:"
    ),
    input_variables=["context", "question"]
)

llm_chain = LLMChain(prompt=prompt, llm=huggingface_llm)

# Input schema
class LLMQuery(BaseModel):
    question: str

@app.post("/llm_query")
def llm_query(query: LLMQuery):
    # Generate embeddings and search in Qdrant
    question_embeddings = MODEL.encode(query.question).tolist()
    search_result = client.search(
        collection_name="similar_text",
        query_vector=question_embeddings,
        limit=1
    )
    context = " ".join([item.payload["text"] for item in search_result]) if search_result else ""

    # Generate the answer
    raw_answer = llm_chain.run(context=context, question=query.question)

    # Trim the output to isolate the relevant text between the first and second occurrence of "Question"
    first_question_index = raw_answer.find("Question")
    second_question_index = raw_answer.find("Question", first_question_index + 1)

    if first_question_index != -1 and second_question_index != -1:
        clean_answer = raw_answer[first_question_index + len("Question:"):second_question_index].strip()
    else:
        # Fallback if structure is not as expected
        clean_answer = raw_answer.strip()

    # Return the response
    response = {
        "question": query.question,
        "context": context,
        "answer": clean_answer
    }
    return JSONResponse(content=response, headers={"Content-Type": "application/json"})

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

C:\Users\thana\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:     Started server process [13368]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57305 - "POST /llm_query HTTP/1.1" 200 OK
INFO:     127.0.0.1:57404 - "POST /llm_query HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13368]


KeyboardInterrupt: 

In [ ]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from transformers import AutoTokenizer, pipeline
from transformers import Qwen2ForCausalLM
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

import uvicorn

app = FastAPI()

# Initialize SentenceTransformer for embeddings
MODEL = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Initialize Qdrant client
client = QdrantClient(url="http://localhost:6333")

# Initialize Qwen2 model for text generation
model_name = "Qwen/Qwen2-0.5B-Chat"  # Update with the appropriate Qwen model name
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = Qwen2ForCausalLM.from_pretrained(model_name, trust_remote_code=True)
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,  # Allows for longer answers
    temperature=0.5,
    repetition_penalty=1.3
)

# Input schema
class LLMQuery(BaseModel):
    question: str

@app.post("/llm_query")
def llm_query(query: LLMQuery):
    # Step 1: Embed the query question
    question_embeddings = MODEL.encode(query.question).tolist()

    # Step 2: Search for similar context in Qdrant
    search_result = client.search(
        collection_name="similar_text",  # Replace with your collection name
        query_vector=question_embeddings,
        limit=3  # Retrieve the top 1 result
    )

    # Extract context from Qdrant results
    context = " ".join([item.payload["text"] for item in search_result]) if search_result else "No relevant context found."

    # Debugging: Log the context and question
    print(f"Context: {context}")
    print(f"Question: {query.question}")

    # Step 3: Prepare the prompt for Qwen2
    prompt = (
        f"Given the following context, provide a short answer to the question.\n\n"
        f"Context:\n{context}\n\n"
        f"Question:\n{query.question}\n\n"
        f"Answer:"
    )

    # Step 4: Generate an answer using the Qwen2 model
    raw_answer = generator(prompt, return_full_text=False)[0]["generated_text"]

    # Debugging: Log the raw answer
    print(f"Raw Answer: {raw_answer}")

    # Extract the answer from the model output
    clean_answer = raw_answer.strip()

    # Step 5: Return the response
    response = {
        "question": query.question,
        "context": context,
        "answer": clean_answer or "No answer generated."
    }
    return JSONResponse(content=response, headers={"Content-Type": "application/json"})

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

C:\Users\thana\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
INFO:     Started server process [18272]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Context: A DECADE OF REOCCURRING SOFTWARE WEAKNESSES- The MDSE scoreThe MDSE equation was designed to balance the frequency and severity in ranking the CWEs. The frequency isdetermined by the number of CVEs that map to a given CWE in the time period of study. The severity is determined by the mean CVSS score for the CVEs mapped to a given CWE. The MDSE score for a CWE is produced by multiplying the normalized frequency by the normalized severity and then multiplying by 100. A DECADE OF REOCCURRING SOFTWARE WEAKNESSES- The MSSW score  Addressing the MDSE's limitationsTo address MDSE limitation 2, the MSSW normalizes the severity using the maximum and minimum mean severityvalues. This gives the distribution a full 0 to 1 range, which is not achieved in the MDSE equation. A DECADE OF REOCCURRING SOFTWARE WEAKNESSES- The MDSE score  Limitation 2: Normalization Error The normalization of the CVSS score does not lead to the expected and desired normalized distribution from 0 to 1. For our da

Context: Heartbleed Revisited: Is it just a Buffer Over-ReadAttacks exploiting Heartbleed, can reveal highly sensitive information - such as private keys, user logincredentials, and business or personal information - via reads over the buffer bounds (for example buffer over-reads). Heartbleed's name literally means a server (or a client) with a vulnerable Heartbeat Extension can "bleed" data via heartbeat response messages. A small heartbeat message with a large requested length can reveal up to 64KB raw memory, and multiple requests can accumulate huge amounts of data. Heartbleed Revisited: Is it just a Buffer Over-ReadHeartbleed was a serious vulnerability in the popular OpenSSL cryptographic software library. The fatal bug was in the Heartbeat Extension of the TLS (Transport Layer Security) protocol implementation. The vulnerability was disclosed in April 2014 with the following Common Vulnerabilities and Exposures (CVE) entry. Heartbleed Revisited: Is it just a Buffer Over-Read -DI

Raw Answer:  Improper Data Value
Explanation:

In this situation, we have three types of errors - under-range, overflowed, and flipped signs.
Under-Range Error occurs when the proper value falls below the maximum allowed value, such as 0x7fffffff. For example,
```
int x = 5;
if(x > INT_MAX)
    printf("Overflow!");
else 
    // code...
```


Over-Range Error occurs when the proper value goes beyond the minimum allowed value, like 4294967295. This can happen because there was no limit set on how large integers could be before Java introduced longs with a maximum length of 32-bit unsigned integer (longLong).
Fliped SignError occurs when you apply a bitwise operator without considering whether your current boolean expression evaluates correctly. In other words, if `a` equals false then `(b & ~(~a)) == b`. If not, the program will crash since any non-zero output means "bad" logic.
Wrong ArgumentsValueError occurs when trying to use invalid arguments during calculations. It happens when att